### Copyright Matlantis Corp. as contributors to Matlantis contrib project

[JP]
##  ポリマーの表面への接着力評価
- シミュレーションはこの[論文](https://pubs.acs.org/doi/10.1021/acs.langmuir.5c03183)を参考に実装した。
- 本notebookでは表面への分子の吸着構造探索を行う。

[EN]
## Evaluation of Polymer Adhesion to a Surface
- This notebook is an implementation based on the following [paper](https://pubs.acs.org/doi/10.1021/acs.langmuir.5c03183).
- In this notebook, we will search for the adsorption structures of molecules on the surface.

In [1]:
import io
import os

from mp_api.client import MPRester
import optuna
from rdkit import Chem

from ase.io import read, write
from ase.constraints import FixAtoms
from ase.optimize import LBFGS

from pymatgen.io.ase import AseAtomsAdaptor
from pymatgen.symmetry.analyzer import SpacegroupAnalyzer

import pfp_api_client
from pfp_api_client.pfp.calculators.ase_calculator import ASECalculator
from pfp_api_client.pfp.estimator import Estimator, EstimatorCalcMode

from pfcc_extras import show_gui
from pfcc_extras.adsorption.adsorption_structure_search import adstructure_search_for_slab
from pfcc_extras.structure.ase_rdkit_converter import smiles_to_atoms
from pfcc_extras.structure.surface import makesurface


def download_structure(mp_id, api_key):
    """
    Downloads the structure from Materials Project for the given MP-ID.

    Parameters:
        mp_id (str): The Materials Project ID of the structure.
        api_key (str): Your Materials Project API key.

    Returns:
        structure (pymatgen.core.structure.Structure): The pymatgen Structure object.
    """
    # Initialize MPRester with your API key
    
    with MPRester(api_key) as m:
        try:
            # Get the structure by MP-ID
            structure = m.get_structure_by_material_id(mp_id, conventional_unit_cell=True)
            atoms = AseAtomsAdaptor.get_atoms(structure)
            print(f"Successfully downloaded structure for MP-ID: {mp_id}")
            return atoms
        except Exception as e:
            print(f"Error downloading structure for MP-ID {mp_id}: {e}")

def to_conventional(atoms):
    """
    Converts an ASE Atoms object to its conventional cell structure.

    Parameters:
        atoms (ase.atoms.Atoms): The input ASE Atoms object, typically
                                 representing a primitive cell.

    Returns:
        ase.atoms.Atoms: A new ASE Atoms object representing the
                         conventional cell of the input structure.
    """
    structure = AseAtomsAdaptor.get_structure(atoms)
    analyzer = SpacegroupAnalyzer(structure)
    conventional_structure = analyzer.get_conventional_standard_structure()
    return AseAtomsAdaptor.get_atoms(conventional_structure)

In [ ]:
# parameter setting
calc_mode="PBE_PLUS_D3"
model_version="v8.0.0"

api_key = "your_api_key"
mp_id = "mp-81"

In [ ]:
outpath = f"output/{mp_id}"  
os.makedirs(outpath, exist_ok=True)

estimator = Estimator(calc_mode=calc_mode, model_version=model_version)
calculator = ASECalculator(estimator)

In [ ]:
# create surface
rep_x, rep_y =  3,3
atoms = download_structure(mp_id, api_key)
surface = to_conventional(atoms)
surface = makesurface(surface, miller_indices=(1,1,1), layers=4, rep=[rep_x, rep_y,1], vacuum=23)
surface.set_tags(1)
show_gui(surface, ball_size=1)

In [ ]:
# set FiaAtoms constraint
thresh = 6.0 # This value was determined by eye from the paper.
constraint = FixAtoms(mask=surface.positions[:, 2] < thresh)
surface.set_constraint(constraint)
write(outpath+"/surface.xyz", surface)

In [ ]:
# set calculator and optimize structure
surface.calc = calculator

opt = LBFGS(surface, logfile=None)
opt.run(fmax=0.01)

In [ ]:
# creare polymer structure (1 unit)
polymer_smiles = "CC(C#N)(C(=O)OCC)CC"
Chem.MolFromSmiles(polymer_smiles)

In [ ]:
# create polymer atoms and opt the structure
polymer = smiles_to_atoms(polymer_smiles)
polymer.set_tags(2)
polymer.calc = calculator
opt = LBFGS(polymer, logfile=None)
opt.run(fmax=0.01)

In [ ]:
show_gui(polymer, representations=["ball+stick"])

In [ ]:
write(outpath+"/polymer.xyz", polymer)

In [ ]:
adstructure_search_for_slab(
    calc_mode       = calc_mode,
    model_version   = model_version,
    molec_path      = outpath+"/polymer.xyz",
    slab_path       = outpath+"/surface.xyz",
    z_height_margin = 1.0,    # set margin between slab and molec.
    fix_slab_thickness = thresh,       # set fix slab thickness. 
    TH_max_f        = 5.0,      
    TH_min_f        = 0.0005,   
    tol             = 0.5, 
    rep_x           = rep_x,
    rep_y           = rep_y,
    zgap            = 2.0,    
    fmax            = 0.01,
    optimizer       = LBFGS,
    output_path     = outpath,
    n_trials        = 1000,
    njobs           = 20,
    sampler         = optuna.samplers.TPESampler(prior_weight=0.5, n_startup_trials=200),
)

[I 2025-10-09 01:54:56,106] Trial 0 finished with value: 0.0 and parameters: {'phi': -0.18038950918575436, 'theta': -0.3905765449981371, 'psi': 0.31981773046606743, 'x_pos': 0.5895358887666047, 'y_pos': 0.4067816353051974, 'z_hig': 3.076560192237687}. Best is trial 0 with value: 0.0.


[I 2025-10-09 01:54:56,138] Trial 17 finished with value: 0.0 and parameters: {'phi': -0.3854072966876281, 'theta': -0.6123164231240086, 'psi': -0.4783231121877749, 'x_pos': 0.41047051134019735, 'y_pos': 0.4552335399470745, 'z_hig': 3.5341353141706997}. Best is trial 0 with value: 0.0.


[I 2025-10-09 01:54:56,155] Trial 3 finished with value: 0.0 and parameters: {'phi': -0.38705982453060916, 'theta': 0.003151223394986191, 'psi': -0.6199914955068981, 'x_pos': 0.6263248518247069, 'y_pos': 0.5760154954007096, 'z_hig': 4.145793381309241}. Best is trial 0 with value: 0.0.


[I 2025-10-09 01:54:56,183] Trial 5 finished with value: 0.0 and parameters: {'phi': 0.23459034296264814, 'theta': -0.1852710092448584, 'psi': 0.9671535536696443, 'x_pos': 0.4642323009343361, 'y_pos': 0.6086888026082145, 'z_hig': 4.6336122227536745}. Best is trial 0 with value: 0.0.


[I 2025-10-09 01:54:56,311] Trial 16 finished with value: 0.0 and parameters: {'phi': -0.48230832643599086, 'theta': -0.11986672004205157, 'psi': -0.6998666186710987, 'x_pos': 0.6620621425198381, 'y_pos': 0.4437945718241241, 'z_hig': 4.366304586122098}. Best is trial 0 with value: 0.0.


...

[I 2025-10-09 03:29:36,161] Trial 994 finished with value: -0.8533846307441308 and parameters: {'phi': -0.3192623256560406, 'theta': -0.2673074899233723, 'psi': 0.705162859607806, 'x_pos': 0.36767887644851416, 'y_pos': 0.6515199416891057, 'z_hig': 4.339301488919431}. Best is trial 516 with value: -1.144414368283421.


[I 2025-10-09 03:29:37,012] Trial 995 finished with value: -0.8834668108065955 and parameters: {'phi': -0.34601363826488984, 'theta': -0.262071313268869, 'psi': 0.7097391488626423, 'x_pos': 0.3900781494212532, 'y_pos': 0.6473693423660262, 'z_hig': 4.365242206956775}. Best is trial 516 with value: -1.144414368283421.


[I 2025-10-09 03:29:39,508] Trial 997 finished with value: -0.8672948446065902 and parameters: {'phi': -0.3289669977575459, 'theta': -0.26930784955990916, 'psi': 0.700849275270112, 'x_pos': 0.3918837369195376, 'y_pos': 0.6309785911903937, 'z_hig': 4.228655700507769}. Best is trial 516 with value: -1.144414368283421.


[I 2025-10-09 03:29:46,241] Trial 999 finished with value: -0.8910698673650899 and parameters: {'phi': -0.3297831572451911, 'theta': -0.26861087105225623, 'psi': 0.703064682574366, 'x_pos': 0.39412314743120513, 'y_pos': 0.630493424041003, 'z_hig': 4.3623452303558095}. Best is trial 516 with value: -1.144414368283421.


[I 2025-10-09 03:30:13,861] Trial 998 finished with value: -0.9605128963570166 and parameters: {'phi': -0.32513026720932725, 'theta': -0.27224827576307153, 'psi': 0.7082376184284993, 'x_pos': 0.39269356810603867, 'y_pos': 0.6306700902487711, 'z_hig': 4.365038457476516}. Best is trial 516 with value: -1.144414368283421.


In [ ]:
# visualize opt history
storage = optuna.storages.JournalStorage(
    optuna.storages.JournalFileStorage(outpath+"/optuna_study.log"),)
study = optuna.load_study(study_name='study',storage=storage)
optuna.visualization.plot_optimization_history(study)

In [ ]:
# view optimized adsorption structure 
adst = read(io.StringIO(study.best_trial.user_attrs["structure"]), format="json")
show_gui(adst, representations=["ball+stick"])

In [2]:
%load_ext watermark
%watermark -n -u -v -iv -w

Last updated: Tue Oct 14 2025

Python implementation: CPython
Python version       : 3.11.11
IPython version      : 9.4.0

rdkit         : 2024.9.4
pfcc_extras   : 0.12.0
pfp_api_client: 1.24.0
ase           : 3.25.0
mp_api        : 0.41.2
optuna        : 4.5.0
pymatgen      : 2025.4.10

Watermark: 2.5.0

